In [ ]:
from ftplib import FTP
import utils
import os
import xml.etree.ElementTree as ET
import glob

server='lhcftp.nlm.nih.gov'
dirs_from_to=(1, 110)
dest_dir='Dataset'
download_imgs=False


source_dirs=['Open-Access-Datasets/Pills/PillProjectDisc' + str(x) +'/images' for x in range(dirs_from_to[0], dirs_from_to[1]+1)]
with FTP(server) as ftp:
    print('Login into {}'.format(server))
    ftp.login()

    base_dir = ftp.pwd()
    
    for idx, source_dir in enumerate(source_dirs):
        ftp.cwd(base_dir)
        ftp.cwd(source_dir)
        
        if not os.path.isdir(os.path.join(dest_dir, str(idx))):
            os.mkdir(os.path.join(dest_dir, str(idx)))
        
        dst = os.path.join(dest_dir, str(idx))

        print('Current woring directory:', ftp.pwd())
        print('Downloading tmp xml')
        filename = 'images.xml'
        with open(os.path.join(filename), 'wb') as f:
            ftp.retrbinary('RETR ' + filename, f.write)

        tree = ET.parse(filename)
        root = tree.getroot()
        
        se = list(root)[0]
        
        print('Filtering xml (' + str(len(list(se))) + ' images)')
        images = []
        for e in list(se):
            layout = e.find('Layout')
            if layout is not None and layout.text == "MC_C3PI_REFERENCE_SEG_V1.6":
                images.append(e.find('File').find('Name').text)
            else:
                se.remove(e)
        
        print('saving xml in:', os.path.join(dst, filename))
        tree.write(os.path.join(dst, filename))
        
        print("final images:", len(images))


        # downloading
        if download_imgs:
            for img in images:
                if os.path.exists(os.path.join(dest_dir, img)):
                    print(img, 'already downloaded!')
                    continue
                with open(os.path.join(dest_dir, img), 'wb') as f:
                    print('downloading ' + img + '...', end='\t')
                    ftp.retrbinary('RETR ' + img, f.write)
    # removing the tmp xml
    os.remove(filename)

In [ ]:
# in this part I check if there are more images with the same type of drug 
dirs = os.listdir(dest_dir)

ids = dict()

expected_size = 0
for d in dirs:
    tree = ET.parse(os.path.join(dest_dir, d, 'images.xml'))
    se = list(tree.getroot())[0]

    for e in list(se):
        expected_size += int(e.find('File').find('Size').text)
        
        i = e.find('MedicosConsultantsID').text
        if i not in ids:
            ids[i] = []
        ids[i].append(e.find('File').find('Name').text) 

sizes = dict()
for _, e in ids.items():
    if len(e) not in sizes:
        sizes[len(e)] = []
    sizes[len(e)].extend(e)

for k in sizes:
    print('There are', len(sizes[k]), 'ids that have', k, 'images associated with them')

print(utils.bytes2human(expected_size), 'will be needed to download all the images')